In [8]:
from bs4 import BeautifulSoup
import zipfile
import os.path
import glob
from chardet.universaldetector import UniversalDetector

In [9]:
def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        for member in zf.infolist():
            # Path traversal defense copied from
            # http://hg.python.org/cpython/file/tip/Lib/http/server.py#l789
            words = member.filename.split('/')
            path = dest_dir
            for word in words[:-1]:
                while True:
                    drive, word = os.path.splitdrive(word)
                    head, word = os.path.split(word)
                    if not drive:
                        print("no drive")
                        break
                if word in (os.curdir, os.pardir, ''):
                    continue
                path = os.path.join(path, word)
            zf.extract(member, path)

In [10]:
def load_txt(dir):
    for path in glob.glob('/' + dir + '/*.txt'):
        with open(path) as markup:
            soup = BeautifulSoup(markup.read(),"html5lib" )
        with open("strip_" + path, "w") as f:
            f.write(soup.get_text().encode('ISO-8859-2'))

In [11]:
def detect(f):
    """Function to detect the right charset in the textfile"""
    detector = UniversalDetector()
    for filename in glob.glob(f):
        print(filename.ljust(60),detector.reset())
        for line in file(filename, 'rb'):
            detector.feed(line)
            if detector.done:
                break
    
    detector.close()
    print(detector.result)

In [12]:
schiller = './txt/schiller.txt'
goethe1 = './txt/goethe1.txt'
goethe2 = './txt/goethe2.txt'

authors = [schiller, goethe1, goethe2]
for txt in authors:
    open(txt)
    detect(txt)
    #soup = BeautifulSoup(f.read(),"html5lib" )
    #g = soup.get_text().encode('ISO-8859-2')

('./txt/schiller.txt                                          ', None)
{'confidence': 0.818998663630605, 'encoding': 'ISO-8859-2'}
('./txt/goethe1.txt                                           ', None)
{'confidence': 0.8284653609588577, 'encoding': 'ISO-8859-2'}
('./txt/goethe2.txt                                           ', None)
{'confidence': 0.99, 'encoding': 'utf-8'}
